# Tutorial para acceder a Minio desde Python

In [1]:
import minio
from minio import Minio
import pandas as pd
from io import BytesIO
import imageio.v3 as iio
import boto3
import os
from dotenv import load_dotenv

Instanciamos un cliente a partir de tokens entregados por el administrador de Minio 

In [4]:
# access = os.getenv('ACCESS')
# secret = os.getenv('SECRET')

access = "FpsOSzUHmLXdb5RL6Szl"
secret = "mICc1qksOuZmX2IRJHUAYRSItDBXXYXaPSwz14Zt"

# Create client with access key and secret key with specific region.
client = Minio(
    "192.168.1.4:9000",
    access_key=access,
    secret_key=secret,
    secure=False
)

## Lectura de archivos 

Usamos el método get para leer un archivo csv desde un bucket llamado raw

In [6]:
# Leer un archivo csv con pandas
obj = client.get_object(
    "imagenes-gee",
    "prueba_descarga_all_bands.csv",
)
df = pd.read_csv(obj, sep=",", nrows = 100000)

df.head()


,Unnamed: 0,B1_mean,B2_mean,B3_mean,B4_mean,B5_mean,B6_mean,B7_mean,B8_mean,B8A_mean,...,MSK_SNWPRB_mean,QA10_mean,QA20_mean,QA60_mean,NDVI_mean,BARREL_SOIL_mean,BUILD_INDEX_mean,Dscrptn,group_id,Name
0,0,2481.196126,2342.950298,2491.999415,2678.286685,2936.119853,3036.452413,3099.815926,3038.988190,3105.063692,...,0.0,0,0,0,0.096070,0.099009,0.076277,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",1,CERRO NAVIA
1,1,1817.432694,1661.999275,1853.696866,2141.707346,2474.961796,2769.417766,2932.116966,2919.550001,3033.322917,...,0.0,0,0,0,0.219092,0.053564,-0.028628,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",1,CERRO NAVIA
2,2,2534.598362,2307.018789,2431.680118,2597.142525,2878.816326,3000.022894,3074.783798,2958.576458,3073.255743,...,0.0,0,0,0,0.097790,0.094358,0.077932,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",1,CERRO NAVIA
3,3,2177.620704,1985.577513,2185.368316,2342.321194,2693.851192,3021.529479,3167.272683,3111.390472,3238.859689,...,0.0,0,0,0,0.185504,0.046619,-0.012230,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",1,CERRO NAVIA
4,4,2068.034239,1861.515815,2058.401085,2242.555645,2612.130296,2946.104467,3099.492652,3070.916059,3183.702790,...,0.0,0,0,0,0.203167,0.042914,-0.033409,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",1,CERRO NAVIA


## Escritura de archivos

Para escribir, usamos el método put

In [7]:

# Crear una variable nueva
nuevo_archivo = df.to_csv().encode("utf-8")

# Escribir con el método put
client.put_object(
    "imagenes-gee",
    "prueba_escritura.csv",
    data=BytesIO(nuevo_archivo),
    length=len(nuevo_archivo),
    content_type='application/csv'
)


Para escribir un archivo parquet procedemos de la misma manera

In [24]:

nuevo_archivo_parquet = df.to_parquet()

client.put_object(
    "raw",
    "archivo_escrito_desde_python.parquet",
    data=BytesIO(nuevo_archivo_parquet),
    length=len(nuevo_archivo_parquet),
    content_type='application/parquet'
)


El procedimiento para escribir un feather es un poco distinto

In [25]:


# Instanciamos una variable que recibirá el feather 
feather_output = BytesIO()

# Escribimos el feather dentro de la instancia de bytes que ya creamos
df.to_feather(feather_output)

# Obtenemos la cantidad de bytes
nb_bytes = feather_output.tell()

# Para buscar la posición cero del archivo
feather_output.seek(0)


client.put_object(
    "raw",
    "archivo_escrito_desde_python.feather",
    data=feather_output,
    length= nb_bytes
)




Para leer un archivo feather o parquet es más sencillo utilizar boto.
Boto no maneja automaticamente el protocolo s3, por lo que debemos indicarlo en el código  

In [26]:
my_bucket_name = 'raw'
my_file_path = 'archivo_escrito_desde_python.parquet'
 
s3Client = boto3.client('s3',endpoint_url='http://192.168.1.4:9000', 
                        aws_access_key_id=access,
                        aws_secret_access_key=secret)

# Leemos con el método get
obj = s3Client.get_object(Bucket=my_bucket_name, Key=my_file_path) 

# Para que funcione la lectura, tenemos que convertir el archivo original en un "flujo de bytes" que entra en read_parquet
df=pd.read_parquet(BytesIO(obj['Body'].read()))

# Revisar
df.head()





,REGION,PROVINCIA,COMUNA,DC,AREA,ZC_LOC,ID_ZONA_LOC,NVIV,NHOGAR,PERSONAN,...,P11PAIS_GRUPO,P12PAIS_GRUPO,ESCOLARIDAD,P16A_GRUPO,REGION_15R,PROVINCIA_15R,COMUNA_15R,P10COMUNA_15R,P11COMUNA_15R,P12COMUNA_15R
0,15,152,15202,1,2,6,13225,1,1,1,...,998,998,4,2,15,152,15202,98,15101,98
1,15,152,15202,1,2,6,13225,3,1,1,...,998,998,0,2,15,152,15202,98,98,98
2,15,152,15202,1,2,6,13225,3,1,2,...,998,998,0,2,15,152,15202,98,98,98
3,15,152,15202,1,2,6,13225,3,1,3,...,998,998,2,2,15,152,15202,98,98,98
4,15,152,15202,1,2,6,13225,3,1,4,...,998,998,3,2,15,152,15202,98,98,98
